This notebook queries the wunderground api for station ids within a 40 km range of Florence, Stevensville Montana.  This information is then used to query weather data for each station.

#### api reference page :
https://www.wunderground.com/weather/api/d/docs?d=resources/code-samples

In [105]:
from urllib.request import urlopen
import json
import pandas as pd

In [ ]:
private_key = 'x'

The following api calls return all weather stations within 40km of 'MT/town.json'

In [106]:
main_call = 'http://api.wunderground.com/api/' + private_key + '/geolookup/conditions/q/'
hamilton_query = 'MT/Hamilton.json'
florence_query = 'MT/Florence.json'
stevi_query = 'MT/Stevensville.json'

In [107]:
def api_call(url):
    f = urlopen(url)
    json_string = f.read() 
    parsed_json = json.loads(json_string)
    return parsed_json

In [108]:
florence_json = api_call(main_call + florence_query)
hamilton_json = api_call(main_call + hamilton_query)
stevi_json = api_call(main_call + stevi_query)

In [109]:
nearby_florence = florence_json['location']['nearby_weather_stations']
nearby_hamilton = hamilton_json['location']['nearby_weather_stations']
nearby_stevi = stevi_json['location']['nearby_weather_stations']

In [110]:
len(nearby_stevi['pws']['station'])

26

In [111]:
airports = []

In [112]:
airports.append(nearby_hamilton['airport']['station'][1])
airports.append(nearby_hamilton['airport']['station'][3])

In [113]:
airports

[{'city': 'Missoula',
  'country': 'US',
  'icao': 'KMSO',
  'lat': '46.91999817',
  'lon': '-114.08999634',
  'state': 'MT'},
 {'city': 'Lowell',
  'country': 'US',
  'icao': 'KP69',
  'lat': '46.13999939',
  'lon': '-115.59999847',
  'state': 'ID'}]

In [114]:
florence_pws = nearby_florence['pws']['station']
stevi_pws = nearby_stevi['pws']['station']
hamilton_pws = nearby_hamilton['pws']['station']

In [115]:
broot_pws = florence_pws + hamilton_pws + stevi_pws

In [116]:
broot_pws[0]

{'city': 'Florence',
 'country': 'US',
 'distance_km': 1,
 'distance_mi': 0,
 'id': 'KMTFLORE12',
 'lat': 46.642807,
 'lon': -114.084076,
 'neighborhood': 'Florence west side',
 'state': 'MT'}

In [117]:
# put list of dictionaries into a dataframe
df = pd.DataFrame(broot_pws)

In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 9 columns):
city            73 non-null object
country         73 non-null object
distance_km     73 non-null int64
distance_mi     73 non-null int64
id              73 non-null object
lat             73 non-null float64
lon             73 non-null float64
neighborhood    73 non-null object
state           73 non-null object
dtypes: float64(2), int64(2), object(5)
memory usage: 5.2+ KB


In [119]:
df = df.drop_duplicates(subset='id', keep='first')

In [120]:
df = df.sort_values(by=['city']).reset_index(drop=True)

In [1]:
df.head()

NameError: name 'df' is not defined

In [122]:
# stations within 40km
import folium

station_map = folium.Map(
    width=900, height=900,
    location=[46.574031, -114.006500],
    zoom_start=9,
    tiles='Stamen Terrain'
)

for index, row in df.iterrows():
    popup_text = row['neighborhood'] + '</br>' + row['id']
    marker = folium.Marker(location=[row['lat'], row['lon']], popup=popup_text)
    marker.add_to(station_map)

station_map

In [123]:
# output complete lookup csv
dirname = '../output/'
filename = 'geolookup-mso_stev_ham.csv'
outputfile = dirname + filename

df.to_csv(outputfile, index=False)

In [135]:
ids = df['id']
ids

0      KMTCORVA9
1     KMTCORVA10
2     KMTCORVA23
3     KMTCORVA12
4     KMTCORVA15
5     KMTCORVA21
6     KMTFLORE12
7      KMTFLORE4
8      KMTFLORE5
9      KMTFLORE7
10     KMTFLORE2
11     KMTFLORE1
12    KMTFLORE17
13    KMTFLORE18
14    KMTFLORE13
15    KMTFLORE11
16     KMTFLORE3
17    KMTHAMIL12
18         MK6S5
19         M6S59
20    KMTHAMIL16
21     KMTHAMIL1
22    KMTHAMIL13
23     KMTHAMIL7
24     KMTHAMIL4
25     KMTLEISU2
26      KMTLOLO3
27      KMTLOLO7
28      KMTLOLO8
29    KMTMISSO44
30    KMTMISSO62
31    KMTMISSO50
32    KMTMISSO59
33    KMTMISSO41
34    KMTMISSO60
35    KMTMISSO15
36    KMTMISSO74
37    KMTSTEVE12
38    KMTSTEVE16
39    KMTSTEVE13
40     KMTSTEVE8
41     KMTVICTO9
Name: id, dtype: object

In [133]:


# output station IDs
dirname = '../output/'
filename = 'station_id-list.csv'

outputfile = dirname + filename
ids.to_csv(outputfile, index=False)